<h2 style='color:purple' align='center'>Naive Bayes Tutorial Part 1: Predicting survival from titanic crash</h2>

<img src="pic1.png" style='height:200px;width:350px'>

<img src="pic2.png" style='height:200px;width:350px'>

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("titanic.csv")
df.head()

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


In [3]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


In [4]:
inputs = df.drop('Survived',axis='columns')
target = df.Survived

In [5]:
#inputs.Sex = inputs.Sex.map({'male': 1, 'female': 2})

In [6]:
dummies = pd.get_dummies(inputs.Sex)
dummies.head(3)

,female,male
0,0,1
1,1,0
2,1,0


In [7]:
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0


**I am dropping male column as well because of dummy variable trap theory. One column is enough to repressent male vs female**

In [8]:
inputs.drop(['Sex','male'],axis='columns',inplace=True)
inputs.head(3)

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1


In [9]:
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [10]:
inputs.Age[:10]

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [11]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head()

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1
3,1,35.0,53.1000,1
4,3,35.0,8.0500,0


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.3)

In [13]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [14]:
model.fit(X_train,y_train)

GaussianNB()

In [15]:
model.score(X_test,y_test)

0.8283582089552238

In [16]:
X_test[0:10]

,Pclass,Age,Fare,female
487,1,58.0,29.7000,0
721,3,17.0,7.0542,0
542,3,11.0,31.2750,1
292,2,36.0,12.8750,0
291,1,19.0,91.0792,1
175,3,18.0,7.8542,0
441,3,20.0,9.5000,0
327,2,36.0,13.0000,1
833,3,23.0,7.8542,0
843,3,34.5,6.4375,0


In [17]:
y_test[0:10]

487    0
721    0
542    0
292    0
291    1
175    0
441    0
327    1
833    0
843    0
Name: Survived, dtype: int64

In [18]:
model.predict(X_test[0:10])

array([0, 0, 1, 0, 1, 0, 0, 1, 0, 0], dtype=int64)

In [19]:
model.predict_proba(X_test[:10])

array([[0.75362953, 0.24637047],
       [0.94039732, 0.05960268],
       [0.29065852, 0.70934148],
       [0.91862505, 0.08137495],
       [0.01342579, 0.98657421],
       [0.94234035, 0.05765965],
       [0.94579387, 0.05420613],
       [0.27399125, 0.72600875],
       [0.94906063, 0.05093937],
       [0.95666666, 0.04333334]])

**Calculate the score using cross validation**

In [20]:
from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(),X_train, y_train, cv=5)

array([0.784     , 0.752     , 0.76      , 0.71774194, 0.76612903])